In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

In [0]:
data = pd.read_csv('/content/drive/My Drive/Text_classification/author_identification_challenge /xtrain_obfuscated.txt', header=None, names=['Text']) 

In [0]:
len(data)

32513

In [0]:
data_labels=pd.read_csv('/content/drive/My Drive/Text_classification/author_identification_challenge /ytrain.txt', header=None, names=['class'])

In [0]:
data_labels.columns

Index(['class'], dtype='object')

In [0]:
#splitting the dataset into training and test validation set
from sklearn import model_selection
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(data['Text'], data_labels['class'], 
                                                                      random_state=2, stratify=data_labels['class'])

**Using TF-IDF vectors on character level using Trigram and bigram**


In [0]:
from sklearn.feature_extraction.text import  CountVectorizer,TfidfVectorizer

#tried using the countvectorizer method accuracy seems to be poor as it tend to ignore the sequence of the letters
#Since it is obfuscated data creating the features based on each character using ngram techniques
#count_vect = CountVectorizer(analyzer='char', token_pattern=r'\w{1,}')

#xtrain_count =  count_vect.fit_transform(X_train)
#xvalid_count = count_vect.transform(X_valid)
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
xtrain_count =  tfidf_vect_ngram_chars.fit_transform(X_train)
xvalid_count = tfidf_vect_ngram_chars.transform(X_valid)

In [0]:
#one hot encoding of the class labels
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(data_labels['class'])
ytrain_count = encoder.transform(y_train)
yvalid_count = encoder.transform(y_valid)

In [0]:
num_labels = 12 #number of class labels
batch_size = 128
xtrain_count.shape[1]

3277

In [0]:
model = Sequential()
model.add(Dense(1024, input_shape=(xtrain_count.shape[1],)))#choosing the number of neurons as 2/3* input shape
model.add(Activation('relu'))
model.add(Dropout(0.5))# drop out layers to avoid overfitting
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))#number of neurons equivalent to output
model.add(Activation('softmax'))#multi class text classification


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 1024)              3356672   
_________________________________________________________________
activation_41 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
activation_42 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_43 (Dense)             (None, 1024)              1049600   
__________

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('nn_weights.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [0]:
history = model.fit(xtrain_count, ytrain_count,
                    batch_size=batch_size,
                    epochs=50,callbacks=[checkpoint],
                    verbose=1,
                    validation_split=0.1)

Train on 21945 samples, validate on 2439 samples
Epoch 1/50
21945/21945 [==============================] - 5s 221us/step - loss: 1.6430 - acc: 0.4410 - val_loss: 1.0847 - val_acc: 0.6212

Epoch 00001: val_acc improved from -inf to 0.62116, saving model to nn_weights.hdf5
Epoch 2/50
21945/21945 [==============================] - 3s 139us/step - loss: 1.0111 - acc: 0.6602 - val_loss: 0.8282 - val_acc: 0.7241

Epoch 00002: val_acc improved from 0.62116 to 0.72407, saving model to nn_weights.hdf5
Epoch 3/50
21945/21945 [==============================] - 3s 135us/step - loss: 0.8260 - acc: 0.7283 - val_loss: 0.7777 - val_acc: 0.7396

Epoch 00003: val_acc improved from 0.72407 to 0.73965, saving model to nn_weights.hdf5
Epoch 4/50
21945/21945 [==============================] - 3s 136us/step - loss: 0.7283 - acc: 0.7574 - val_loss: 0.7069 - val_acc: 0.7618

Epoch 00004: val_acc improved from 0.73965 to 0.76179, saving model to nn_weights.hdf5
Epoch 5/50
21945/21945 [==========================

In [0]:
score = model.evaluate(xvalid_count, yvalid_count,
                       batch_size=batch_size, verbose=1)

print('Test accuracy:', score[1])


8129/8129 [==============================] - 0s 51us/step
Test accuracy: 0.7692213065364085


In [0]:
predictions = model.predict(xvalid_count)
pred = predictions.argmax(axis=-1)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
cm = confusion_matrix(y_valid, pred)
cm_df = pd.DataFrame(cm.T, index=encoder.classes_, columns=encoder.classes_)
cm_df.index.name = 'Predicted'
cm_df.columns.name = 'True'
print(cm_df)

True        0    1    2    3    4    5    6     7    8    9    10   11
Predicted                                                             
0          117    0    1    2    1    0    0     3    0    0    0    0
1            1  661    9   48    2    1   60    19   22    8   17   15
2            2    9  261   14    4    5   15    11    2   20   19   22
3            4   79   25  785   13    9   20    78   17   18  107   20
4            0    0    1    3  439    6    3     2   15    2    2    1
5            2    2    1    9   15  531    1     9    8    1    4    7
6            0   48   17   18    6    2  808    39   24    9   43   20
7            6   22   10   33   24    4   50  1023    9    9   43   18
8            1    8    3    9   50    6   25    16  769    0   17    8
9            0    4    2    7    0    0    3     1    0  148    0    2
10           2   21   25   69   29    2   48    45   36   25  488   16
11           1   11   13    9    1    5   24    28    6    5   23  223


In [0]:
print(classification_report(y_valid,pred)) 

              precision    recall  f1-score   support

           0       0.92      0.78      0.84       136
           1       0.82      0.71      0.76       865
           2       0.72      0.71      0.72       368
           3       0.71      0.76      0.74      1006
           4       0.85      0.78      0.82       584
           5       0.91      0.92      0.91       571
           6       0.71      0.81      0.76      1057
           7       0.80      0.80      0.80      1274
           8       0.81      0.86      0.83       908
           9       0.77      0.71      0.74       245
          10       0.64      0.61      0.63       763
          11       0.66      0.60      0.63       352

   micro avg       0.77      0.77      0.77      8129
   macro avg       0.78      0.75      0.76      8129
weighted avg       0.77      0.77      0.77      8129



In [0]:
X_test = pd.read_csv('/content/drive/My Drive/Text_classification/author_identification_challenge /xtest_obfuscated.txt', names = ['Text'])

In [0]:

xtest_count =  tfidf_vect_ngram_chars.transform(X_test['Text'])

In [0]:
xtest_count.shape

(3000, 3277)

In [0]:
pred_test = model.predict(xtest_count)
pred_label = pred_test.argmax(axis=-1)

In [0]:
X_test['label'] = pred_label

In [0]:
X_test.to_csv('/content/drive/My Drive/Text_classification/author_identification_challenge /output.csv', sep='\t', encoding='utf-8')